# 自动语音识别

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/TksaY_FDgnk?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

自动语音识别（ASR）将语音信号转换为文本，将一系列音频输入映射到文本输出。
Siri 和 Alexa 这类虚拟助手使用 ASR 模型来帮助用户日常生活，还有许多其他面向用户的有用应用，如会议实时字幕和会议纪要。

本指南将向您展示如何：

1. 在 [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14) 数据集上对
   [Wav2Vec2](https://huggingface.co/facebook/wav2vec2-base) 进行微调，以将音频转录为文本。
2. 使用微调后的模型进行推断。

<Tip>

如果您想查看所有与本任务兼容的架构和检查点，最好查看[任务页](https://huggingface.co/tasks/automatic-speech-recognition)。

</Tip>

在开始之前，请确保您已安装所有必要的库：

```bash
pip install transformers datasets evaluate jiwer
```

我们鼓励您登录自己的 Hugging Face 账户，这样您就可以上传并与社区分享您的模型。
出现提示时，输入您的令牌登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 加载 MInDS-14 数据集

首先从🤗 Datasets 库中加载 [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14)
数据集的一个较小子集。这将让您有机会先进行实验，确保一切正常，然后再花更多时间在完整数据集上进行训练。

In [ ]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

使用 `~Dataset.train_test_split` 方法将数据集的 `train` 拆分为训练集和测试集：

In [ ]:
minds = minds.train_test_split(test_size=0.2)

然后看看数据集：

In [ ]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 16
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 4
    })
})

虽然数据集包含 `lang_id `和 `english_transcription` 等许多有用的信息，但在本指南中，
您将专注于 `audio` 和 `transcription`。使用 `remove_columns` 方法删除其他列：

In [ ]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

再看看示例：

In [ ]:
minds["train"][0]

{'audio': {'array': array([-0.00024414,  0.        ,  0.        , ...,  0.00024414,
          0.00024414,  0.00024414], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~APP_ERROR/602ba9e2963e11ccd901cd4f.wav',
  'sampling_rate': 8000},
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~APP_ERROR/602ba9e2963e11ccd901cd4f.wav',
 'transcription': "hi I'm trying to use the banking app on my phone and currently my checking and savings account balance is not refreshing"}

有 2 个字段：

- `audio`：由语音信号形成的一维 `array`，用于加载和重新采样音频文件。
- `transcription`：目标文本。

## 预处理

下一步是加载一个 Wav2Vec2 处理器来处理音频信号：

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

MInDS-14 数据集的采样率为 8000kHz（您可以在其[数据集卡片](https://huggingface.co/datasets/PolyAI/minds14)中找到此信息），
这意味着您需要将数据集重新采样为 16000kHz 以使用预训练的 Wav2Vec2 模型：

In [ ]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'audio': {'array': array([-2.38064706e-04, -1.58618059e-04, -5.43987835e-06, ...,
          2.78103951e-04,  2.38446111e-04,  1.18740834e-04], dtype=float32),
  'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~APP_ERROR/602ba9e2963e11ccd901cd4f.wav',
  'sampling_rate': 16000},
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~APP_ERROR/602ba9e2963e11ccd901cd4f.wav',
 'transcription': "hi I'm trying to use the banking app on my phone and currently my checking and savings account balance is not refreshing"}

如您在上面的 `transcription` 中所看到的，文本包含大小写字符的混合。
Wav2Vec2 分词器仅训练了大写字符，因此您需要确保文本与分词器的词汇表匹配：

In [ ]:
def uppercase(example):
    return {"transcription": example["transcription"].upper()}


minds = minds.map(uppercase)

现在创建一个预处理函数，该函数应该：

1. 调用 `audio` 列以加载和重新采样音频文件。
2. 从音频文件中提取 `input_values` 并使用处理器对 `transcription` 列执行 tokenizer 操作。

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"] = len(batch["input_values"][0])
    return batch

要在整个数据集上应用预处理函数，可以使用🤗 Datasets 的 `map` 函数。
您可以通过增加 `num_proc` 参数来加速 `map` 的处理进程数量。
使用 `remove_columns` 方法删除不需要的列：

In [ ]:
encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

🤗 Transformers 没有用于 ASR 的数据整理器，因此您需要调整 [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/zh/main_classes/data_collator#transformers.DataCollatorWithPadding) 来创建一个示例批次。
它还会动态地将您的文本和标签填充到其批次中最长元素的长度（而不是整个数据集），以使它们具有统一的长度。
虽然可以通过在 `tokenizer` 函数中设置 `padding=True` 来填充文本，但动态填充更有效。

与其他数据整理器不同，这个特定的数据整理器需要对 `input_values` 和 `labels `应用不同的填充方法：

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: list[dict[str, Union[list[int], torch.Tensor]]]) -> dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

现在实例化您的 `DataCollatorForCTCWithPadding`：

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

## 评估

在训练过程中包含一个指标通常有助于评估模型的性能。
您可以通过🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) 库快速加载一个评估方法。
对于这个任务，加载 [word error rate](https://huggingface.co/spaces/evaluate-metric/wer)（WER）指标
（请参阅🤗 Evaluate [快速上手](https://huggingface.co/docs/evaluate/a_quick_tour)以了解如何加载和计算指标）：

In [ ]:
import evaluate

wer = evaluate.load("wer")

然后创建一个函数，将您的预测和标签传递给 `compute` 来计算 WER：

In [ ]:
import numpy as np


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

您的 `compute_metrics` 函数现在已经准备就绪，当您设置好训练时将返回给此函数。

## 训练

<Tip>

要深入了解如何微调模型进行自动语音识别，
请查看这篇博客[文章](https://huggingface.co/blog/fine-tune-wav2vec2-english)以了解英语 ASR，
还可以参阅[这篇文章](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2)以了解多语言 ASR。

</Tip>

## 推断

很好，现在您已经微调了一个模型，您可以用它进行推断了！

加载您想要运行推断的音频文件。请记住，如果需要，将音频文件的采样率重新采样为与模型匹配的采样率！

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", "en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

尝试使用微调后的模型进行推断的最简单方法是使用 [pipeline()](https://huggingface.co/docs/transformers/main/zh/main_classes/pipelines#transformers.pipeline)。
使用您的模型实例化一个用于自动语音识别的 `pipeline`，并将您的音频文件传递给它：

In [ ]:
from transformers import pipeline

transcriber = pipeline("automatic-speech-recognition", model="stevhliu/my_awesome_asr_minds_model")
transcriber(audio_file)

{'text': 'I WOUD LIKE O SET UP JOINT ACOUNT WTH Y PARTNER'}

<Tip>

转录结果还不错，但可以更好！尝试用更多示例微调您的模型，以获得更好的结果！

</Tip>

如果您愿意，您也可以手动复制 `pipeline` 的结果：